# 휴먼지능정보공학과 201810808 정민지


## 연습문제: 은행 송금



은행은 자신이 계정을 개설하여, 입출금을 할 수 있다.

컨트랙을 프로그램하여 아래 기능을 실행하도록 하세요.

주피터 노트북으로 프로그램하세요.



은행 컨트랙의 함수는 다음과 같이 만드세요.

* 입금액을 정해서 입금

	function deposit(uint amount) public payable

* 전액 인출 함수

	function widthdrawAll() public

* 컨트랙 잔고확인 함수 (this를 이용한 잔고, 상태변수 잔고 (this 잔고가 맞는지 확인하는 용도)

	function getBalance() public view returns(uint, uint)

* 다른 계정으로 계좌이체 함수

	function forwardTo(address payable _receiver) public payable



아래 기능을 실행해 보세요.

- 입금 11111 wei, 222 wei를 하고 잔고 11333 wei 출력

- 자신의 2번째 계정으로 계좌이체 333 wei하고 컨트랙 잔고 11000 wei 출력, 자신의 2번째 계정 잔고 증가분 (+333 wei) 출력

- 전액 인출하고 11000 wei, 자신의 잔고 증가분 출력 (+11000 wei)

In [579]:
%%writefile src/Bank.sol
pragma solidity 0.5.1;
contract Bank {
    
    address payable myAddr1;
    address payable myAddr2;
    uint balance;
    uint balNow;
    uint init1;
    uint init2;
    
    constructor() public {
        balance = address(this).balance;
    }
    
    function setAddr1(address payable addr1) public {
        myAddr1=addr1;
    }
    
    function setAddr2(address payable addr2) public {
        myAddr2=addr2;
    }
    
    function deposit(uint amount) public payable{
        require(msg.value == amount); //require: 예외발생구문
        balance += amount;
    }
    
    function widthdrawAll() public{
        balNow=address(this).balance;
        balance -= balNow;
        require(myAddr1.transfer(balNow));
    }
    
    function getBalance() public view returns(uint, uint){
        return (address(this).balance,balance); 
    }
    
  
    function forwardTo(address payable _receiver) public payable{
        balance -= 333;
        require(_receiver.send(333));
    }
    
    function balanceOfAccount1() public view returns (uint) {
        return myAddr1.balance;
    }
    
    function balanceOfAccount2() public view returns (uint) {
        return myAddr2.balance;
    }
    
    function setInit1() public {
        init1=myAddr1.balance;
    }
    
    function subtract1() public view returns (uint) {
        return myAddr1.balance-init1;
    }
    
    function setInit2() public {
        init2=myAddr2.balance;
    }
    
    function subtract2() public view returns (uint) {
        return myAddr2.balance-init2;
    }
    
}

Overwriting src/Bank.sol


In [522]:
!solc --gas --abi --bin src/Bank.sol


======= src/Bank.sol:Bank =======
Gas estimation:
construction:
   20787 + 327000 = 347787
external:
   balanceOfAccount1():	992
   balanceOfAccount2():	860
   deposit(uint256):	20646
   forwardTo(address):	infinite
   getBalance():	835
   setAddr1(address):	20704
   setAddr2(address):	20726
   setInit1():	20827
   setInit2():	20915
   subtract1():	1154
   subtract2():	1220
   widthdrawAll():	infinite
Binary: 
608060405234801561001057600080fd5b503073ffffffffffffffffffffffffffffffffffffffff16316002819055506106638061003e6000396000f3fe6080604052600436106100b4576000357c01000000000000000000000000000000000000000000000000000000009004806312065fe0146100b957806312fa11a5146100eb5780631c1921f11461011657806327d8ad881461012d5780633c4593751461017157806341f89b3b146101885780638ea51007146101b3578063a0532697146101ca578063aba11bc5146101f5578063b6b55f2514610220578063e6cee3eb1461024e578063eb183bf61461029f575b600080fd5b3480156100c557600080fd5b506100ce6102f0565b60405180838152602001828152602001925050506040518

In [671]:
%%writefile src/BankDeploy.js
var Web3=require('web3');
var web3;
if (typeof web3 !== 'undefined') {
    web3 = new Web3(web3.currentProvider);
} else {
    web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8445"));  //geth
}
var _abiArray=[{"constant":true,"inputs":[],"name":"getBalance","outputs":[{"name":"","type":"uint256"},{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"balanceOfAccount2","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"setInit1","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"_receiver","type":"address"}],"name":"forwardTo","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[],"name":"widthdrawAll","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"subtract1","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"setInit2","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"balanceOfAccount1","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"subtract2","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[{"name":"addr1","type":"address"}],"name":"setAddr1","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"addr2","type":"address"}],"name":"setAddr2","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"}];
var _bin="608060405234801561001057600080fd5b503073ffffffffffffffffffffffffffffffffffffffff16316002819055506106638061003e6000396000f3fe6080604052600436106100b4576000357c01000000000000000000000000000000000000000000000000000000009004806312065fe0146100b957806312fa11a5146100eb5780631c1921f11461011657806327d8ad881461012d5780633c4593751461017157806341f89b3b146101885780638ea51007146101b3578063a0532697146101ca578063aba11bc5146101f5578063b6b55f2514610220578063e6cee3eb1461024e578063eb183bf61461029f575b600080fd5b3480156100c557600080fd5b506100ce6102f0565b604051808381526020018281526020019250505060405180910390f35b3480156100f757600080fd5b50610100610316565b6040518082815260200191505060405180910390f35b34801561012257600080fd5b5061012b610357565b005b61016f6004803603602081101561014357600080fd5b81019080803573ffffffffffffffffffffffffffffffffffffffff169060200190929190505050610398565b005b34801561017d57600080fd5b506101866103ef565b005b34801561019457600080fd5b5061019d610484565b6040518082815260200191505060405180910390f35b3480156101bf57600080fd5b506101c86104c8565b005b3480156101d657600080fd5b506101df61050a565b6040518082815260200191505060405180910390f35b34801561020157600080fd5b5061020a61054a565b6040518082815260200191505060405180910390f35b61024c6004803603602081101561023657600080fd5b810190808035906020019092919050505061058f565b005b34801561025a57600080fd5b5061029d6004803603602081101561027157600080fd5b81019080803573ffffffffffffffffffffffffffffffffffffffff1690602001909291905050506105b0565b005b3480156102ab57600080fd5b506102ee600480360360208110156102c257600080fd5b81019080803573ffffffffffffffffffffffffffffffffffffffff1690602001909291905050506105f3565b005b6000803073ffffffffffffffffffffffffffffffffffffffff1631600254915091509091565b6000600160009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1631905090565b6000809054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1631600481905550565b61014d6002600082825403925050819055508073ffffffffffffffffffffffffffffffffffffffff166108fc61014d9081150290604051600060405180830381858888f1935050505015156103ec57600080fd5b50565b3073ffffffffffffffffffffffffffffffffffffffff16316003819055506003546002600082825403925050819055506000809054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff166108fc6003549081150290604051600060405180830381858888f19350505050151561048257600080fd5b565b60006004546000809054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff163103905090565b600160009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1631600581905550565b60008060009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1631905090565b6000600554600160009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff163103905090565b803414151561059d57600080fd5b8060026000828254019250508190555050565b806000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff16021790555050565b80600160006101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff1602179055505056fea165627a7a72305820bb57cb48da054a6ace582e35b220d965620757b2ffc35d81b3238a8d27ed8d5c0029";
var _contract = new web3.eth.Contract(_abiArray);
_contract
    .deploy({data:"0x"+_bin})
    .send({from: "0x8e71085ffefccb88a4b54d4c93a866db7928a486", gas: 2000000, gasPrice: '100000'})
    .then(function(newContractInstance){
        console.log(newContractInstance.options.address) // instance with the new contract address
    });

Overwriting src/BankDeploy.js


In [794]:
!node src/BankDeploy.js

0xE79602b619b6e0E72C2c94e3e5bbAf267B8c2e31


###  입금 11111 wei, 222 wei를 하고 

In [845]:
%%writefile src/BankUse.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8445"));
var _abiArray=[{"constant":true,"inputs":[],"name":"getBalance","outputs":[{"name":"","type":"uint256"},{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"balanceOfAccount2","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"setInit1","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"_receiver","type":"address"}],"name":"forwardTo","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[],"name":"widthdrawAll","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"subtract1","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"setInit2","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"balanceOfAccount1","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"subtract2","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[{"name":"addr1","type":"address"}],"name":"setAddr1","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"addr2","type":"address"}],"name":"setAddr2","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"}];
var myBank = new web3.eth.Contract(_abiArray, '0xE79602b619b6e0E72C2c94e3e5bbAf267B8c2e31');

myBank.methods.setAddr1("0x9eabca74ecd2d27638af3a8ee7532b7853a0310d").send({from:"0x8e71085ffefccb88a4b54d4c93a866db7928a486",gas:80000});
myBank.methods.setAddr2("0x223bd75a28f01aad68f309fce60a22b06252d420").send({from:"0x8e71085ffefccb88a4b54d4c93a866db7928a486",gas:80000});

myBank.methods.deposit(11111).send({from:"0x8e71085ffefccb88a4b54d4c93a866db7928a486",gas:80000,value:11111});
myBank.methods.deposit(222).send({from:"0x8e71085ffefccb88a4b54d4c93a866db7928a486",gas:80000,value:222});

Overwriting src/BankUse.js


In [846]:
!node src/BankUse.js

In [847]:
%%writefile src/BankUse2.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8445"));
var _abiArray=[{"constant":true,"inputs":[],"name":"getBalance","outputs":[{"name":"","type":"uint256"},{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"balanceOfAccount2","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"setInit1","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"_receiver","type":"address"}],"name":"forwardTo","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[],"name":"widthdrawAll","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"subtract1","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"setInit2","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"balanceOfAccount1","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"subtract2","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[{"name":"addr1","type":"address"}],"name":"setAddr1","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"addr2","type":"address"}],"name":"setAddr2","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"}];
var myBank = new web3.eth.Contract(_abiArray, '0xE79602b619b6e0E72C2c94e3e5bbAf267B8c2e31');

myBank.methods.getBalance().call().then(function(value) {console.log('My Contract: ', value);});

Overwriting src/BankUse2.js


### 잔고 11333 wei 출력

In [848]:
!node src/BankUse2.js

My Contract:  Result { '0': '11333', '1': '11333' }


### 자신의 2번째 계정으로 계좌이체 333 wei하고(forwardTo함수)

In [849]:
%%writefile src/BankUse3.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8445"));
var _abiArray=[{"constant":true,"inputs":[],"name":"getBalance","outputs":[{"name":"","type":"uint256"},{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"balanceOfAccount2","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"setInit1","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"_receiver","type":"address"}],"name":"forwardTo","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[],"name":"widthdrawAll","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"subtract1","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"setInit2","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"balanceOfAccount1","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"subtract2","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[{"name":"addr1","type":"address"}],"name":"setAddr1","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"addr2","type":"address"}],"name":"setAddr2","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"}];
var myBank = new web3.eth.Contract(_abiArray, '0xE79602b619b6e0E72C2c94e3e5bbAf267B8c2e31');

myBank.methods.balanceOfAccount1().call().then(function(value) {console.log('balance of Account 1: ', value);});
myBank.methods.balanceOfAccount2().call().then(function(value) {console.log('balance of Account 2: ', value);});

myBank.methods.setInit2().send({from:"0x8e71085ffefccb88a4b54d4c93a866db7928a486",gas:80000});

myBank.methods.forwardTo("0x223bd75a28f01aad68f309fce60a22b06252d420").send({from:"0x8e71085ffefccb88a4b54d4c93a866db7928a486",gas:80000});



Overwriting src/BankUse3.js


In [850]:
!node src/BankUse3.js

balance of Account 1:  25000000000000207000
balance of Account 2:  55000000000000006660


In [851]:
%%writefile src/BankUse4.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8445"));
var _abiArray=[{"constant":true,"inputs":[],"name":"getBalance","outputs":[{"name":"","type":"uint256"},{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"balanceOfAccount2","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"setInit1","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"_receiver","type":"address"}],"name":"forwardTo","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[],"name":"widthdrawAll","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"subtract1","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"setInit2","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"balanceOfAccount1","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"subtract2","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[{"name":"addr1","type":"address"}],"name":"setAddr1","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"addr2","type":"address"}],"name":"setAddr2","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"}];
var myBank = new web3.eth.Contract(_abiArray, '0xE79602b619b6e0E72C2c94e3e5bbAf267B8c2e31');

myBank.methods.getBalance().call().then(function(value) {console.log('My Contract: ', value);});
myBank.methods.subtract2().call().then(function(value) {console.log('increase of second account: ',value);});

myBank.methods.balanceOfAccount1().call().then(function(value) {console.log('balance of Account 1: ', value);});
myBank.methods.balanceOfAccount2().call().then(function(value) {console.log('balance of Account 2: ', value);});

Overwriting src/BankUse4.js


### 컨트랙 잔고 11000 wei 출력, 자신의 2번째 계정 잔고 증가분 (+333 wei) 출력

In [852]:
!node src/BankUse4.js

increase of second account:  333
My Contract:  Result { '0': '11000', '1': '11000' }
balance of Account 1:  25000000000000207000
balance of Account 2:  55000000000000006993


### 전액 인출하고(widthdrawAll)

In [853]:
%%writefile src/BankUse5.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8445"));
var _abiArray=[{"constant":true,"inputs":[],"name":"getBalance","outputs":[{"name":"","type":"uint256"},{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"balanceOfAccount2","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"setInit1","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"_receiver","type":"address"}],"name":"forwardTo","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[],"name":"widthdrawAll","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"subtract1","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"setInit2","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"balanceOfAccount1","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"subtract2","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[{"name":"addr1","type":"address"}],"name":"setAddr1","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"addr2","type":"address"}],"name":"setAddr2","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"}];
var myBank = new web3.eth.Contract(_abiArray, '0xE79602b619b6e0E72C2c94e3e5bbAf267B8c2e31');

myBank.methods.balanceOfAccount1().call().then(function(value) {console.log('balance of Account 1: ', value);});
myBank.methods.balanceOfAccount2().call().then(function(value) {console.log('balance of Account 2: ', value);});


myBank.methods.setInit1().send({from:"0x8e71085ffefccb88a4b54d4c93a866db7928a486",gas:80000});

myBank.methods.widthdrawAll().send({from:"0x8e71085ffefccb88a4b54d4c93a866db7928a486",gas:80000});


Overwriting src/BankUse5.js


In [854]:
!node src/BankUse5.js

balance of Account 1:  25000000000000207000
balance of Account 2:  55000000000000006993


In [855]:
%%writefile src/BankUse6.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8445"));
var _abiArray=[{"constant":true,"inputs":[],"name":"getBalance","outputs":[{"name":"","type":"uint256"},{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"balanceOfAccount2","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"setInit1","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"_receiver","type":"address"}],"name":"forwardTo","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[],"name":"widthdrawAll","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"subtract1","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"setInit2","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"balanceOfAccount1","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"subtract2","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[{"name":"addr1","type":"address"}],"name":"setAddr1","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"addr2","type":"address"}],"name":"setAddr2","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"}];
var myBank = new web3.eth.Contract(_abiArray, '0xE79602b619b6e0E72C2c94e3e5bbAf267B8c2e31');

myBank.methods.getBalance().call().then(function(value) {console.log('My Contract: ', value);});
myBank.methods.subtract1().call().then(function(value) {console.log('increase of myaccount: ',value);});

myBank.methods.balanceOfAccount1().call().then(function(value) {console.log('balance of Account 1: ', value);});
myBank.methods.balanceOfAccount2().call().then(function(value) {console.log('balance of Account 2: ', value);});


Overwriting src/BankUse6.js


### 자신의 잔고 증가분 출력 (+11000 wei)

In [856]:
!node src/BankUse6.js

My Contract:  Result { '0': '0', '1': '0' }
increase of myaccount:  11000
balance of Account 1:  25000000000000218000
balance of Account 2:  55000000000000006993
